# AWS Bedrock Agents with Couchbase Vector Search - Lambda Approach

This notebook demonstrates the Lambda approach for implementing AWS Bedrock agents with Couchbase Vector Search. In this approach, the agent invokes AWS Lambda functions to execute operations.

We'll implement a multi-agent architecture with specialized agents for different tasks:
- **Researcher Agent**: Searches for relevant documents in the vector store
- **Writer Agent**: Formats and presents the research findings

## Alternative Approaches

This notebook demonstrates the Lambda Approach for AWS Bedrock Agents. For comparison, you might also want to check out the Custom Control Approach, which handles agent tools directly in your application code instead of using AWS Lambda functions.

The Custom Control approach offers simpler setup and more direct control, but may not scale as well. You can find that implementation here: [Custom Control Approach Notebook](../custom-control-approach/Bedrock_Agents_Custom_Control.ipynb)

Note: If the link above doesn't work in your Jupyter environment, you can navigate to the file manually in the `awsbedrock-agents/custom-control-approach/` directory.

## Overview

The Lambda approach delegates the execution of an agent's defined functions (tools) to separate AWS Lambda functions. When the agent decides to use a tool, Bedrock directly invokes the corresponding Lambda function specified in the action group configuration, passing the required parameters. The Lambda function executes the logic and returns the result to the agent, which then continues processing within the same invocation.

## Key Steps & Concepts

1.  **Define Agent:**
    *   Define instructions (prompt) for the agent.
    *   Define the function schema (tools the agent can use, e.g., `researcher_functions_lambda`, `writer_functions_lambda` in the example). The schema defines the interface the agent expects.

2.  **Implement Lambda Handlers:**
    *   Create separate AWS Lambda functions to implement the logic for each tool defined in the schema (e.g., one Lambda for `search_documents`, another for `format_content`).
    *   These Lambda functions receive an event payload containing the API path, parameters, and other details of the agent's request.
    *   The Lambda code needs to parse this event, execute the required action (e.g., query the vector store, format text), and return a specific JSON response structure that Bedrock expects.
    *   The example includes a `deploy` script in `awsbedrock-agents/lambda_functions` to package and deploy these handlers.
    *   *Environment Configuration:* The Lambdas need access to necessary resources and configuration (e.g., Couchbase connection details, Bedrock embeddings client). The example writes a `.env` file before deployment to potentially pass this information, though the exact mechanism within the Lambda deployment/runtime isn't fully shown.

3.  **Create Agent in Bedrock:**
    *   Use `bedrock_agent_client.create_agent` similar to the Custom Control approach.

4.  **Create Action Group (Lambda):**
    *   Use `bedrock_agent_client.create_agent_action_group`.
    *   Crucially, set the `actionGroupExecutor` to `{"lambda": "arn:aws:lambda:<region>:<account_id>:function:<lambda_function_name>"}`. This points Bedrock to the specific Lambda function ARN responsible for executing the actions in this group.
    *   Provide the `functionSchema` defined earlier.

5.  **Prepare Agent:**
    *   Use `bedrock_agent_client.prepare_agent`.

6.  **Create Agent Alias:**
    *   Use `bedrock_agent_client.create_agent_alias`.

7.  **Invoke Agent:**
    *   Use `bedrock_runtime_client.invoke_agent`.
    *   Bedrock handles the invocation of the configured Lambda function when the agent decides to use a tool.
    *   The application code simply receives the final response from the agent after the tool execution (if any) is complete. It doesn't need to handle `returnControl` events for function execution.
    *   *Debugging Note:* The example's `invoke_agent` function includes extensive debugging prints and attempts to manually re-execute the logic *if* the agent's final response `result` is empty but a `returnControl` event *was* observed. This suggests potential issues or complexities in correctly receiving the final result after Lambda execution in the streaming response, leading to this fallback/debugging logic being added in the script.

## Pros

*   **Decoupling:** Tool execution logic is separate from the main application, potentially managed by different teams or deployment cycles.
*   **Scalability:** Leverages the inherent scalability and serverless nature of AWS Lambda for tool execution.
*   **Managed Execution:** AWS manages the invocation and execution environment for the Lambda functions.
*   **Simpler Application Code:** The application invoking the agent doesn't need to implement the tool logic or handle the `returnControl` event for execution.

## Cons

*   **Deployment Complexity:** Requires setting up, configuring, and deploying separate Lambda functions, including managing their dependencies and permissions.
*   **State Management:** Passing state or context between the main application and the Lambda functions can be more complex (e.g., requires passing connection details, potentially initializing clients within the Lambda).
*   **Cold Starts:** Lambda cold starts can introduce latency into the agent's response time.
*   **Debugging:** Debugging issues that span the Bedrock Agent service and the Lambda execution can be more challenging.
*   **Cost:** Incurs separate Lambda execution costs. 

# Installing Necessary Libraries
As a prerequisite, we need to install the necessary libraries. We can do this using pip.

In [1]:
%pip install --quiet langchain-couchbase==0.3.0 langchain-aws==0.2.22 couchbase==4.3.5

Note: you may need to restart the kernel to use updated packages.


## Setup and Configuration

First, let's import the necessary libraries and set up our environment:

In [2]:
import json
import logging
import os
import time
import uuid
import subprocess
from datetime import timedelta

import boto3
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.exceptions import (InternalServerFailureException,
                                  QueryIndexAlreadyExistsException,
                                  ServiceUnavailableException)
from couchbase.management.buckets import CreateBucketSettings
from couchbase.management.search import SearchIndex
from couchbase.options import ClusterOptions
from dotenv import load_dotenv
from langchain_aws import BedrockEmbeddings
from langchain_couchbase.vectorstores import CouchbaseSearchVectorStore

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

### Load Environment Variables

Load environment variables from the .env file. Make sure to create a .env file with the necessary credentials before running this notebook.

In [3]:
# Load environment variables
load_dotenv()

# Couchbase Configuration
CB_HOST = os.getenv("CB_HOST", "couchbase://localhost")
CB_USERNAME = os.getenv("CB_USERNAME", "Administrator")
CB_PASSWORD = os.getenv("CB_PASSWORD", "password")
CB_BUCKET_NAME = os.getenv("CB_BUCKET_NAME", "vector-search-testing")
SCOPE_NAME = os.getenv("SCOPE_NAME", "shared")
COLLECTION_NAME = os.getenv("COLLECTION_NAME", "bedrock") 
INDEX_NAME = os.getenv("INDEX_NAME", "vector_search_bedrock")

# AWS Configuration
AWS_REGION = os.getenv("AWS_REGION", "us-east-1")
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_ACCOUNT_ID = os.getenv("AWS_ACCOUNT_ID")

# Check if required environment variables are set
required_vars = ["AWS_ACCESS_KEY_ID", "AWS_SECRET_ACCESS_KEY", "AWS_ACCOUNT_ID"]
missing_vars = [var for var in required_vars if not os.getenv(var)]
if missing_vars:
    print(f"Missing required environment variables: {', '.join(missing_vars)}")
    print("Please set these variables in your .env file")
else:
    print("All required environment variables are set")

All required environment variables are set


### Initialize AWS Clients

Set up the AWS clients for Bedrock and other services:

In [4]:
# Initialize AWS session
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_REGION
)

# Initialize AWS clients from session
iam_client = session.client('iam')
bedrock_client = session.client('bedrock')
bedrock_agent_client = session.client('bedrock-agent')
bedrock_runtime = session.client('bedrock-runtime')
bedrock_runtime_client = session.client('bedrock-agent-runtime')
lambda_client = session.client('lambda')

print("AWS clients initialized successfully")

AWS clients initialized successfully


## Set Up Couchbase and Vector Store

Now let's set up the Couchbase connection, collections, and vector store:

In [5]:
def setup_collection(cluster, bucket_name, scope_name, collection_name):
    """Set up Couchbase collection"""
    try:
        # Check if bucket exists, create if it doesn't
        try:
            bucket = cluster.bucket(bucket_name)
            print(f"Bucket '{bucket_name}' exists.")
        except Exception as e:
            print(f"Bucket '{bucket_name}' does not exist. Creating it...")
            bucket_settings = CreateBucketSettings(
                name=bucket_name,
                bucket_type='couchbase',
                ram_quota_mb=1024,
                flush_enabled=True,
                num_replicas=0
            )
            cluster.buckets().create_bucket(bucket_settings)
            bucket = cluster.bucket(bucket_name)
            print(f"Bucket '{bucket_name}' created successfully.")

        bucket_manager = bucket.collections()

        # Check if scope exists, create if it doesn't
        scopes = bucket_manager.get_all_scopes()
        scope_exists = any(scope.name == scope_name for scope in scopes)
        
        if not scope_exists and scope_name != "_default":
            print(f"Scope '{scope_name}' does not exist. Creating it...")
            bucket_manager.create_scope(scope_name)
            print(f"Scope '{scope_name}' created successfully.")

        # Check if collection exists, create if it doesn't
        collections = bucket_manager.get_all_scopes()
        collection_exists = any(
            scope.name == scope_name and collection_name in [col.name for col in scope.collections]
            for scope in collections
        )

        if not collection_exists:
            print(f"Collection '{collection_name}' does not exist. Creating it...")
            bucket_manager.create_collection(scope_name, collection_name)
            print(f"Collection '{collection_name}' created successfully.")
        else:
            print(f"Collection '{collection_name}' already exists. Skipping creation.")

        # Wait for collection to be ready
        collection = bucket.scope(scope_name).collection(collection_name)
        time.sleep(2)  # Give the collection time to be ready for queries

        # Ensure primary index exists
        try:
            cluster.query(f"CREATE PRIMARY INDEX IF NOT EXISTS ON `{bucket_name}`.`{scope_name}`.`{collection_name}`").execute()
            print("Primary index present or created successfully.")
        except Exception as e:
            print(f"Error creating primary index: {str(e)}")

        # Clear all documents in the collection
        try:
            query = f"DELETE FROM `{bucket_name}`.`{scope_name}`.`{collection_name}`"
            cluster.query(query).execute()
            print("All documents cleared from the collection.")
        except Exception as e:
            print(f"Error while clearing documents: {str(e)}. The collection might be empty.")

        return collection
    except Exception as e:
        print(f"Error setting up collection: {str(e)}")
        raise

In [6]:
def setup_indexes(cluster):
    """Set up search indexes"""
    try:
        # Load index definition from file
        with open('aws_index.json', 'r') as file:
            index_definition = json.load(file)
            print(f"Loaded index definition from aws_index.json")
    except Exception as e:
        print(f"Error loading index definition: {str(e)}")
        raise
    
    try:
        scope_index_manager = cluster.bucket(CB_BUCKET_NAME).scope(SCOPE_NAME).search_indexes()

        # Check if index already exists
        existing_indexes = scope_index_manager.get_all_indexes()
        index_name = index_definition["name"]

        if index_name in [index.name for index in existing_indexes]:
            print(f"Index '{index_name}' found")
        else:
            print(f"Creating new index '{index_name}'...")

        # Create SearchIndex object from JSON definition
        search_index = SearchIndex.from_json(index_definition)

        # Upsert the index (create if not exists, update if exists)
        scope_index_manager.upsert_index(search_index)
        print(f"Index '{index_name}' successfully created/updated.")

    except QueryIndexAlreadyExistsException:
        print(f"Index '{index_name}' already exists. Skipping creation/update.")
    except ServiceUnavailableException:
        print("Search service is not available. Please ensure the Search service is enabled in your Couchbase cluster.")
    except InternalServerFailureException as e:
        print(f"Internal server error: {str(e)}")
        raise

In [7]:
# Connect to Couchbase
auth = PasswordAuthenticator(CB_USERNAME, CB_PASSWORD)
options = ClusterOptions(auth)
cluster = Cluster(CB_HOST, options)
cluster.wait_until_ready(timedelta(seconds=5))
print("Successfully connected to Couchbase")

# Set up collections
collection = setup_collection(cluster, CB_BUCKET_NAME, SCOPE_NAME, COLLECTION_NAME)
print("Collections setup complete")

# Set up search indexes
setup_indexes(cluster)
print("Search indexes setup complete")

Successfully connected to Couchbase
Bucket 'vector-search-testing' exists.
Collection 'bedrock' already exists. Skipping creation.
Primary index present or created successfully.
All documents cleared from the collection.
Collections setup complete
Loaded index definition from aws_index.json
Index 'vector_search_bedrock' found
Index 'vector_search_bedrock' already exists. Skipping creation/update.
Search indexes setup complete


In [8]:
# Initialize Bedrock runtime client for embeddings
embeddings = BedrockEmbeddings(
    client=bedrock_runtime,
    model_id="amazon.titan-embed-text-v2:0"
)
print("Successfully created Bedrock embeddings client")

# Initialize vector store
vector_store = CouchbaseSearchVectorStore(
    cluster=cluster,
    bucket_name=CB_BUCKET_NAME,
    scope_name=SCOPE_NAME,
    collection_name=COLLECTION_NAME,
    embedding=embeddings,
    index_name=INDEX_NAME
)
print("Successfully created vector store")

Successfully created Bedrock embeddings client
Successfully created vector store


## Load Documents from JSON File

Let's load the documents from the documents.json file:

In [9]:
# Load documents from JSON file
try:
    with open('documents.json', 'r') as f:
        data = json.load(f)
        documents = data.get('documents', [])
    print(f"Loaded {len(documents)} documents from documents.json")
except Exception as e:
    print(f"Error loading documents: {str(e)}")
    raise

# Add documents to vector store
print(f"Adding {len(documents)} documents to vector store...")
for i, doc in enumerate(documents, 1):
    text = doc.get('text', '')
    metadata = doc.get('metadata', {})
    
    # Add document to vector store
    metadata_dict = json.loads(metadata) if isinstance(metadata, str) else metadata or {}
    doc_ids = vector_store.add_texts([text], [metadata_dict])
    doc_id = doc_ids[0] if doc_ids else None    
    print(f"Added document {i}/{len(documents)} with ID: {doc_id}")
    
    # Add small delay between requests
    time.sleep(1)

print(f"\nProcessing complete: {len(documents)}/{len(documents)} documents added successfully")

Loaded 7 documents from documents.json
Adding 7 documents to vector store...
Added document 1/7 with ID: dcbdea3907d54ef0b0ca892f37b9242a
Added document 2/7 with ID: e2b05cc1e26743f3b1d838f60065589c
Added document 3/7 with ID: 0f3f8ee598b140059514d9e625b92777
Added document 4/7 with ID: f39cf196ebab423c9fb8b56dc00cf730
Added document 5/7 with ID: fdb8bb35e38741beb89bfc657578b6b8
Added document 6/7 with ID: ccac0d68aafd4818b85250194bf89cf4
Added document 7/7 with ID: 29b4433ea4e74b6b935545ae150c571a

Processing complete: 7/7 documents added successfully


## Lambda Approach Implementation

Now let's implement the Lambda approach for Bedrock agents. This approach involves deploying Lambda functions that will be invoked by the Bedrock agents.

### Deploy Lambda Functions

First, let's deploy the Lambda functions that will be invoked by our Bedrock agents. We'll create a .env file for the Lambda functions with the Couchbase configuration.

In [10]:
# Create a .env file for the Lambda functions
lambda_env_path = 'lambda_functions/.env'
with open(lambda_env_path, 'w') as f:
    f.write(f"CB_HOST={CB_HOST}\n")
    f.write(f"CB_USERNAME={CB_USERNAME}\n")
    f.write(f"CB_PASSWORD={CB_PASSWORD}\n")
    f.write(f"CB_BUCKET_NAME={CB_BUCKET_NAME}\n")
    f.write(f"SCOPE_NAME={SCOPE_NAME}\n")
    f.write(f"COLLECTION_NAME={COLLECTION_NAME}\n")
    f.write(f"INDEX_NAME={INDEX_NAME}\n")

print(f"Created .env file for Lambda functions at {lambda_env_path}")

Created .env file for Lambda functions at lambda_functions/.env


In [11]:
# Deploy Lambda functions
print("Deploying Lambda functions...")
try:
    subprocess.run([
        'python3', 
        'lambda_functions/deploy.py'
    ], check=True)
    print("Lambda functions deployed successfully")
except subprocess.CalledProcessError as e:
    print(f"Error deploying Lambda functions: {str(e)}")
    raise RuntimeError("Failed to deploy Lambda functions")

Deploying Lambda functions...
Using existing IAM role: bedrock_agent_lambda_role
Deleting existing Lambda function: bedrock_agent_researcher
Waiting for bedrock_agent_researcher to be deleted...
Deleting existing Lambda function: bedrock_agent_writer
Waiting for bedrock_agent_writer to be deleted...

=== Deploying researcher function ===
Installing dependencies for bedrock_agent_researcher...
Waiting for role arn:aws:iam::598307997273:role/bedrock_agent_lambda_role to be ready...
Zip file size: 53.12 MB
File size (53.12 MB) exceeds 10MB. Using S3 for deployment...
Creating S3 bucket: lambda-deployment-598307997273-1745960136 (attempt 1/3)...
Created S3 bucket: lambda-deployment-598307997273-1745960136
Waiting for bucket to be available...
Uploading bedrock_agent_researcher.zip to S3 bucket lambda-deployment-598307997273-1745960136...
Successfully uploaded bedrock_agent_researcher.zip to s3://lambda-deployment-598307997273-1745960136/lambda/bedrock_agent_researcher.zip-ffbce51f
Creating

## Lambda Approach Helper Functions

Let's define some helper functions for the Lambda approach:

In [12]:
def wait_for_agent_status(bedrock_agent_client, agent_id, target_statuses=['Available', 'PREPARED', 'NOT_PREPARED'], max_attempts=30, delay=2):
    """Wait for agent to reach any of the target statuses"""
    for attempt in range(max_attempts):
        try:
            response = bedrock_agent_client.get_agent(agentId=agent_id)
            current_status = response['agent']['agentStatus']
            
            if current_status in target_statuses:
                print(f"Agent {agent_id} reached status: {current_status}")
                return current_status
            elif current_status == 'FAILED':
                print(f"Agent {agent_id} failed")
                return 'FAILED'
            
            print(f"Agent status: {current_status}, waiting... (attempt {attempt + 1}/{max_attempts})")
            time.sleep(delay)
            
        except Exception as e:
            print(f"Error checking agent status: {str(e)}")
            time.sleep(delay)
    
    return current_status

In [13]:
def create_agent(bedrock_agent_client, name, instructions, model_id="amazon.nova-pro-v1:0"):
    """Create a Bedrock agent with Lambda action groups"""
    try:
        # List existing agents
        existing_agents = bedrock_agent_client.list_agents()
        existing_agent = next(
            (agent for agent in existing_agents['agentSummaries'] 
             if agent['agentName'] == name),
            None
        )
        
        # Handle existing agent
        if existing_agent:
            agent_id = existing_agent['agentId']
            print(f"Found existing agent '{name}' with ID: {agent_id}")
            
            # Check agent status
            response = bedrock_agent_client.get_agent(agentId=agent_id)
            status = response['agent']['agentStatus']
            
            if status in ['NOT_PREPARED', 'FAILED']:
                print(f"Deleting agent '{name}' with status {status}")
                bedrock_agent_client.delete_agent(agentId=agent_id)
                time.sleep(10)  # Wait after deletion
                existing_agent = None
        
        # Create new agent if needed
        if not existing_agent:
            print(f"Creating new agent '{name}'")
            agent = bedrock_agent_client.create_agent(
                agentName=name,
                description=f"{name.title()} agent for document operations",
                instruction=instructions,
                idleSessionTTLInSeconds=1800,
                foundationModel=model_id
            )
            agent_id = agent['agent']['agentId']
            print(f"Created new agent '{name}' with ID: {agent_id}")
        else:
            agent_id = existing_agent['agentId']
        
        # Wait for initial creation if needed
        status = wait_for_agent_status(bedrock_agent_client, agent_id, target_statuses=['NOT_PREPARED', 'PREPARED', 'Available'])
        if status not in ['NOT_PREPARED', 'PREPARED', 'Available']:
            raise Exception(f"Agent failed to reach valid state: {status}")
        
        # Handle alias creation/retrieval
        try:
            aliases = bedrock_agent_client.list_agent_aliases(agentId=agent_id)
            alias = next((a for a in aliases['agentAliasSummaries'] if a['agentAliasName'] == 'v1'), None)
            
            if not alias:
                print(f"Creating new alias for agent '{name}'")
                alias = bedrock_agent_client.create_agent_alias(
                    agentId=agent_id,
                    agentAliasName="v1"
                )
                alias_id = alias['agentAlias']['agentAliasId']
            else:
                alias_id = alias['agentAliasId']
                print(f"Using existing alias for agent '{name}'")
            
            print(f"Successfully configured agent '{name}' with ID: {agent_id} and alias: {alias_id}")
            return agent_id, alias_id
            
        except Exception as e:
            print(f"Error managing alias: {str(e)}")
            raise
        
    except Exception as e:
        print(f"Error creating/updating agent: {str(e)}")
        raise RuntimeError(f"Failed to create/update agent: {str(e)}")

In [14]:
# awsbedrock-agents/lamda-approach/Bedrock_Agents_Lambda.ipynb

def invoke_agent(bedrock_runtime_client, agent_id, alias_id, input_text, session_id=None, vector_store=None):
    """Invoke a Bedrock agent with improved debugging and error handling"""
    if session_id is None:
        session_id = str(uuid.uuid4())

    try:
        print(f"--- Invoking agent {agent_id} (Alias: {alias_id}) ---")
        print(f"Session ID: {session_id}")
        print(f"Input Text: '{input_text[:100]}...'\") # Print truncated input")

        response = bedrock_runtime_client.invoke_agent(
            agentId=agent_id,
            agentAliasId=alias_id,
            sessionId=session_id,
            inputText=input_text,
            enableTrace=True
        )

        result = ""
        print("--- Processing Response Stream ---")
        event_count = 0
        lambda_invoked = False
        lambda_result_received = None

        for event in response['completion']:
            event_count += 1
            print(f"\n[Event {event_count}] Type: {list(event.keys())}")

            # Print full event for detailed inspection - Handle potential TypeError
            try:
                # Attempt pretty printing first
                print(f"Event Content:\n{json.dumps(event, indent=2)}")
            except TypeError:
                # Fallback to standard print if JSON serialization fails (e.g., due to datetime)
                print(f"Event Content (raw, contains non-serializable types):\n{event}")


            if 'chunk' in event:
                chunk = event['chunk']['bytes'].decode('utf-8')
                print(f"  -> Received Chunk: '{chunk}'")
                result += chunk

            if 'trace' in event:
                print("  -> Received Trace event.")
                trace_data = event['trace']
                # Log trace details - especially useful for Lambda interactions
                if isinstance(trace_data, dict) and 'orchestrationTrace' in trace_data:
                    orch_trace = trace_data['orchestrationTrace']
                    if 'invocationInput' in orch_trace:
                         print(f"    -> Trace: Invocation Input found (Agent intends to call function)")
                         lambda_invoked = True # Mark that the agent tried to call something
                    if 'invocationOutput' in orch_trace:
                        invocation_output = orch_trace['invocationOutput']
                        if 'actionGroupInvocationOutput' in invocation_output:
                            action_output = invocation_output['actionGroupInvocationOutput']
                            if 'responseBody' in action_output:
                                print("    -> Trace: Lambda Response Body found")
                                response_body = action_output['responseBody']
                                if isinstance(response_body, dict) and 'application/json' in response_body:
                                    json_body = response_body['application/json']
                                    if 'body' in json_body:
                                        lambda_result_received = json_body['body']
                                        print(f"      -> Lambda Result Body: {lambda_result_received}")
                                        # In Lambda mode, Bedrock should ideally incorporate this into the final 'chunk'
                                        # But we capture it here for debugging if the final result is empty
                                        # Assigning here might short-circuit final LLM processing, so use cautiously
                                        # result = lambda_result_received # <-- Potentially uncomment if chunks never arrive after lambda call

            if 'returnControl' in event:
                 # This shouldn't happen often in Lambda mode, but log if it does
                 print("  -> Received returnControl event (unexpected in Lambda mode)")


        print(f"\n--- Stream Processing Complete ({event_count} events) ---")

        if not result.strip():
            print("WARN: Final 'result' is empty or whitespace.")
            if lambda_invoked and lambda_result_received is not None:
                 print("INFO: Lambda was invoked and returned a result, but it wasn't in the final agent output.")
                 print(f"INFO: Raw Lambda Result was: {lambda_result_received}")
                 # Decide if you want to return the raw lambda result as a fallback
                 # return lambda_result_received
            elif lambda_invoked:
                 print("ERROR: Agent trace indicates Lambda invocation, but no result was found in the trace.")
            else:
                 print("INFO: No Lambda invocation was detected in the trace.")


        print(f"--- Returning Result --- \n'{result}'")
        return result

    except Exception as e:
        print(f"--- ERROR during agent invocation ---")
        import traceback
        print(traceback.format_exc()) # Print full exception traceback
        raise RuntimeError(f"Failed to invoke agent: {str(e)}")

## Define Agent Instructions and Functions

Now let's define the instructions and functions for our agents:

In [15]:
# Researcher agent instructions
researcher_instructions = """
You are a Research Assistant that helps users find relevant information in documents.
Your capabilities include:
1. Searching through documents using semantic similarity
2. Providing relevant document excerpts
3. Answering questions based on document content
"""

# Researcher agent functions
researcher_functions = [{
    "name": "search_documents",
    "description": "Search for relevant documents using semantic similarity",
    "parameters": {
        "query": {
            "type": "string",
            "description": "The search query",
            "required": True
        },
        "k": {
            "type": "integer",
            "description": "Number of results to return",
            "required": False
        }
    },
    "requireConfirmation": "DISABLED"
}]

# Writer agent instructions
writer_instructions = """
You are a Content Writer Assistant that helps format and present research findings.
Your capabilities include:
1. Formatting research findings in a user-friendly way
2. Creating clear and engaging summaries
3. Organizing information logically
4. Highlighting key insights
"""

# Writer agent functions
writer_functions = [{
    "name": "format_content",
    "description": "Format and present research findings",
    "parameters": {
        "content": {
            "type": "string",
            "description": "The research findings to format",
            "required": True
        },
        "style": {
            "type": "string",
            "description": "The desired presentation style (e.g., summary, detailed, bullet points)",
            "required": False
        }
    },
    "requireConfirmation": "DISABLED"
}]

## Run Lambda Approach

Now let's run the Lambda approach with our agents:

In [16]:
# Create researcher agent
try:
    researcher_id, researcher_alias = create_agent(
        bedrock_agent_client,
        "researcher", 
        researcher_instructions, 
    )
    print(f"Researcher agent created with ID: {researcher_id} and alias: {researcher_alias}")
except Exception as e:
    print(f"Failed to create researcher agent: {str(e)}")
    researcher_id, researcher_alias = None, None

# Create writer agent
try:
    writer_id, writer_alias = create_agent(
        bedrock_agent_client,
        "writer", 
        writer_instructions, 
    )
    print(f"Writer agent created with ID: {writer_id} and alias: {writer_alias}")
except Exception as e:
    print(f"Failed to create writer agent: {str(e)}")
    writer_id, writer_alias = None, None

if not any([researcher_id, writer_id]):
    raise RuntimeError("Failed to create any agents")

Creating new agent 'researcher'
Created new agent 'researcher' with ID: BQFTVQNMLY
Agent status: CREATING, waiting... (attempt 1/30)
Agent BQFTVQNMLY reached status: NOT_PREPARED
Creating new alias for agent 'researcher'
Error managing alias: An error occurred (ValidationException) when calling the CreateAgentAlias operation: Create operation can't be performed on AgentAlias when Agent is in Not Prepared state.
Error creating/updating agent: An error occurred (ValidationException) when calling the CreateAgentAlias operation: Create operation can't be performed on AgentAlias when Agent is in Not Prepared state.
Failed to create researcher agent: Failed to create/update agent: An error occurred (ValidationException) when calling the CreateAgentAlias operation: Create operation can't be performed on AgentAlias when Agent is in Not Prepared state.
Creating new agent 'writer'
Created new agent 'writer' with ID: HFUSWMZ3MC
Agent status: CREATING, waiting... (attempt 1/30)
Agent HFUSWMZ3MC re

RuntimeError: Failed to create any agents

In [ ]:
# Create action group for researcher agent with Lambda executor
try:
    bedrock_agent_client.create_agent_action_group(
        agentId=researcher_id,
        agentVersion="DRAFT",
        actionGroupExecutor={
            "lambda": f"arn:aws:lambda:{AWS_REGION}:{AWS_ACCOUNT_ID}:function:bedrock_agent_researcher"
        },  # This is the key for Lambda approach
        actionGroupName="researcher_actions",
        functionSchema={"functions": researcher_functions},
        description="Action group for researcher operations with Lambda"
    )
    print("Created researcher Lambda action group")
except bedrock_agent_client.exceptions.ConflictException:
    print("Researcher Lambda action group already exists")
    
# Prepare researcher agent
print("Preparing researcher agent...")
bedrock_agent_client.prepare_agent(agentId=researcher_id)
status = wait_for_agent_status(
    bedrock_agent_client,
    researcher_id, 
    target_statuses=['PREPARED', 'Available']
)
print(f"Researcher agent preparation completed with status: {status}")

In [ ]:
# Create action group for writer agent with Lambda executor
try:
    bedrock_agent_client.create_agent_action_group(
        agentId=writer_id,
        agentVersion="DRAFT",
        actionGroupExecutor={
            "lambda": f"arn:aws:lambda:{AWS_REGION}:{AWS_ACCOUNT_ID}:function:bedrock_agent_writer"
        },  # This is the key for Lambda approach
        actionGroupName="writer_actions",
        functionSchema={"functions": writer_functions},
        description="Action group for writer operations with Lambda"
    )
    print("Created writer Lambda action group")
except bedrock_agent_client.exceptions.ConflictException:
    print("Writer Lambda action group already exists")
    
# Prepare writer agent
print("Preparing writer agent...")
bedrock_agent_client.prepare_agent(agentId=writer_id)
status = wait_for_agent_status(
    bedrock_agent_client,
    writer_id, 
    target_statuses=['PREPARED', 'Available']
)
print(f"Writer agent preparation completed with status: {status}")

## Test the Agents

Let's test our agents by asking the researcher agent to search for information and the writer agent to format the results:

In [ ]:
# Test researcher agent
researcher_response = invoke_agent(
    bedrock_runtime_client,
    researcher_id,
    researcher_alias,
    'What is unique about the Cline AI assistant? Use the search_documents function to find relevant information.',
    vector_store=vector_store
)
print("\nResearcher Response:\n", researcher_response)

In [ ]:
# Test writer agent
writer_response = invoke_agent(
    bedrock_runtime_client,
    writer_id,
    writer_alias,
    f'Format this research finding using the format_content function: {researcher_response}',
    vector_store=vector_store
)
print("\nWriter Response:\n", writer_response)

## Conclusion

In this notebook, we've demonstrated the Lambda approach for implementing AWS Bedrock agents with Couchbase Vector Search. This approach allows the agent to invoke AWS Lambda functions to execute operations, providing better scalability and separation of concerns.

Key components of this implementation include:

1. **Vector Store Setup**: We set up a Couchbase vector store to store and search documents using semantic similarity.
2. **Lambda Function Deployment**: We deployed Lambda functions that handle the agent's function calls.
3. **Agent Creation**: We created two specialized agents - a researcher agent for searching documents and a writer agent for formatting results.
4. **Lambda Integration**: We integrated the agents with Lambda functions, allowing them to execute operations in a serverless environment.

This approach is particularly useful for production environments where scalability and separation of concerns are important. The Lambda functions can be deployed independently and can access other AWS services, providing more flexibility and power.